# ClimbBot IK test notebook

This notebook provides a runnable test for inverse kinematics (IK) on your 2R+prismatic arm in the MJCF file you provided. It includes:

- Analytic 2R+prismatic IK (planar) cell (you need to provide `L1` and `L2` measured from your MJCF).
- Numerical IK that uses `mujoco` (or `mujoco_py`) and `scipy.optimize.least_squares` to solve for joint values that place the end-effector `r_grip_site` at a target position.

Save your MJCF as `climbbot3.xml` in the same folder as this notebook before running the cells. If `mujoco` isn't installed in this environment, the notebook will print instructions to install it locally.

----


## 1) Analytic planar 2R + prismatic IK (use if arm is planar)

Set `L1` and `L2` below (distance between revolute joint axes). This cell computes joint angles for a target in the arm base frame (x,y) and returns `theta1, theta2, d` (d clipped to prismatic limits).


In [2]:
import numpy as np
from math import atan2, sqrt

def analytic_2R_prismatic(px, py, L1, L2, d_min=0.0, d_max=0.134, elbow='down'):
    """
    Solve planar 2R + prismatic. Returns (theta1, theta2, d).
    - px,py: target in arm base frame (meters)
    - L1,L2: link lengths (meters)
    - d_min,d_max: prismatic limits
    - elbow: 'down' or 'up'
    """
    r = np.hypot(px, py)
    cos_th2 = (r*r - L1*L1 - L2*L2) / (2*L1*L2)
    if abs(cos_th2) > 1.0:
        raise ValueError(f"Target out of reach: cos_theta2={cos_th2}")
    if elbow == 'down':
        th2 = atan2(+sqrt(max(0.0, 1 - cos_th2*cos_th2)), cos_th2)
    else:
        th2 = atan2(-sqrt(max(0.0, 1 - cos_th2*cos_th2)), cos_th2)
    th1 = atan2(py, px) - atan2(L2 * np.sin(th2), L1 + L2 * np.cos(th2))
    # Placeholder for d: depends on how your prismatic axis is defined.
    # Here we return a nominal mid-range value; replace with geometry-aware computation.
    d = 0.05
    d = float(np.clip(d, d_min, d_max))
    return float(th1), float(th2), float(d)

# Example usage (replace L1,L2 with measured values from your MJCF):
L1 = 0.08  # meters — REPLACE with your measured distance between r1 and r2 axes
L2 = 0.12  # meters — REPLACE with your measured distance between r2 and the prismatic joint axis
px, py = 0.15, 0.03
try:
    th1, th2, d = analytic_2R_prismatic(px, py, L1, L2)
    print("Analytic IK solution (base frame): theta1, theta2, d =", th1, th2, d)
except Exception as e:
    print("Analytic IK failed:", e)


Analytic IK solution (base frame): theta1, theta2, d = -0.6928696291454255 1.4349623360274977 0.05


## 2) Numerical IK using Mujoco + SciPy
This cell will try to import `mujoco` (new Python bindings). If not present it will try `mujoco_py` fallback. If neither is available, the cell prints install instructions.

The solver optimizes the **independent joint qpos** (r1, r2, r3_1) to minimize the Euclidean error between the `r_grip_site` world position and the target.


In [11]:
# IK test cell tailored for your mujoco environment (uses MjModel.from_xml_path + mj_* API)
import numpy as np
from scipy.optimize import least_squares
import mujoco
import os

# -------------------------
# Helper: name -> id (same style as your env)
# -------------------------
def name2id(model, objtype, name):
    if objtype == 'site':
        return mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, name)
    if objtype == 'joint':
        return mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, name)
    if objtype == 'body':
        return mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, name)
    if objtype == 'actuator':
        return mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, name)
    raise ValueError("unsupported objtype")

# -------------------------
# Config: model path and target
# -------------------------
MODEL_PATH = "/Users/aaronthomas/Desktop/Engineering_Projects/Climbing Robot/assets/robot_mjcf.xml"   # change to your MJCF path if needed
TARGET_WORLD = np.array([0.35, 0.03, 0.35])   # world coordinates target
ARM = "r"   # "r" or "l" arm for which to run IK

# -------------------------
# Load model + data
# -------------------------
assert os.path.exists(MODEL_PATH), f"Model file not found: {MODEL_PATH}"
model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

# -------------------------
# IDs & joint mapping (independent joints)
# -------------------------
site_name = f"{ARM}_grip_site"
site_id = name2id(model, "site", site_name)
base_site_id = name2id(model, "site", "base_site")  # optional

if ARM == "r":
    joint_names = ["r1", "r2", "r3_1"]
    actuator_names = ["r1_ctrl", "r2_ctrl", "r3_1_ctrl"]
else:
    joint_names = ["l1", "l2", "l3_1"]
    actuator_names = ["l1_ctrl", "l2_ctrl", "l3_1_ctrl"]

# joint ids (mj API) and their qpos adr / dof adr
joint_ids = [ name2id(model, "joint", n) for n in joint_names ]
qpos_addrs = [ model.jnt_qposadr[jid] for jid in joint_ids ]
dof_addrs  = [ model.jnt_dofadr[jid] for jid in joint_ids ]   # for Jacobian column selection

# optional: actuator ids if present
actuator_ids = []
for an in actuator_names:
    aid = name2id(model, "actuator", an)
    if aid != -1:
        actuator_ids.append(aid)

print("Using site:", site_name, "site_id:", site_id)
print("Joint qpos addrs:", qpos_addrs, "dof addrs:", dof_addrs)
print("Found actuators:", actuator_ids)

# ensure forward kinematics baseline
mujoco.mj_forward(model, data)
print("Initial site pos:", data.site_xpos[site_id].copy())

# -------------------------
# Pack/unpack q vector helpers
# -------------------------
def pack_full_q(x_local):
    """Given vector x_local of length m (the independent joints in order),
    return a full qpos vector suitable for writing into data.qpos."""
    q = data.qpos.copy()
    for i, qpos_addr in enumerate(qpos_addrs):
        q[qpos_addr] = float(x_local[i])
    return q

def set_q_and_forward(q_full):
    data.qpos[:] = q_full
    mujoco.mj_forward(model, data)

def get_site_pos():
    mujoco.mj_forward(model, data)
    return data.site_xpos[site_id].copy()

# -------------------------
# Residual for least squares (optimize joint qpos of independent joints)
# -------------------------
def residual(x_local, target_world):
    qfull = pack_full_q(x_local)
    set_q_and_forward(qfull)
    site_pos = data.site_xpos[site_id]
    return (site_pos - target_world)

# -------------------------
# Initial guess: current qpos for those joints
# -------------------------
x0 = np.array([ data.qpos[qaddr] for qaddr in qpos_addrs ], dtype=float)
print("Initial guess (qpos):", x0)

# -------------------------
# Run least squares IK
# -------------------------
res = least_squares(residual, x0, args=(TARGET_WORLD,), verbose=2, xtol=1e-7, ftol=1e-7, max_nfev=500)
sol = res.x
print("IK success:", res.success, "message:", res.message)
print("solution qpos (for joints):", sol)

# -------------------------
# Validate apply & report
# -------------------------
qfull_solution = pack_full_q(sol)
set_q_and_forward(qfull_solution)
site_after = get_site_pos()
error_vec = site_after - TARGET_WORLD
print("site after IK:", site_after, "target:", TARGET_WORLD, "error (norm):", np.linalg.norm(error_vec))

# -------------------------
# Optionally: write solution into actuators (if you want to test closed-loop control)
# -------------------------
if len(actuator_ids) == len(sol):
    for aid, val in zip(actuator_ids, sol):
        data.ctrl[aid] = float(val)
    print("Wrote solution to actuators data.ctrl[...]")
else:
    print("Actuator count doesn't match independent joint count — skipping ctrl write.")
    print("You can still step with mj_step() and the qpos will be the current kinematic state if you set data.qpos.")

# Optionally step a few times (if you wrote ctrl targets) to let actuators move:
for _ in range(20):
    mujoco.mj_step(model, data)

print("site after stepping:", data.site_xpos[site_id].copy())


Using site: r_grip_site site_id: 1
Joint qpos addrs: [np.int32(0), np.int32(1), np.int32(2)] dof addrs: [np.int32(0), np.int32(1), np.int32(2)]
Found actuators: [0, 1, 2]
Initial site pos: [ 0.41379051 -0.21070016  0.10983577]
Initial guess (qpos): [0. 0. 0.]
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.9842e-02                                    6.38e-02    
       1              2         2.9266e-02      3.06e-02       1.00e+00       2.19e-01    
       2              3         7.2010e-03      2.21e-02       1.00e+00       2.85e-02    
       3              5         2.0066e-03      5.19e-03       2.90e-01       2.04e-02    
       4              6         2.9557e-04      1.71e-03       4.39e-01       9.53e-03    
       5              7         9.1748e-07      2.95e-04       1.10e-01       9.33e-04    
       6              8         4.9988e-12      9.17e-07       3.18e-03       1.47e-06    
       7    

## 3) Notes and troubleshooting

- If the IK fails to converge: try different initial guesses, reduce target distance, or use analytic IK if the arm is planar.
- If `site` position is far from the target after applying `ctrl` values, the actuators may be too weak or gains too low/high; try setting `sim.data.qpos` directly and `sim.forward()` to validate the kinematic solution.
- Make sure contact/mask issues (contype/conaffinity) are fixed when testing grasping/climbing. For pure IK tests (no contact) you can ignore contact settings.
